In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import lightning.pytorch as pl
import torch

sys.path.insert(1, sys.path[0] + '/..')
from src.data.datamodule import DataModule
from src.model.setup import setup_model

from src.run import parse_option
args = parse_option(notebook=True)

Cell below should be the same as `main()` in `run.py`

In [2]:
# pl.seed_everything(args.seed, workers=True)
# torch.use_deterministic_algorithms(warn_only=True)
torch.set_float32_matmul_precision('medium')

datamodule = DataModule(dir = '../data_dev', num_workers=args.num_workers, batch_size=args.batch_size)
model = setup_model()

trainer = pl.Trainer(
    logger = pl.loggers.TensorBoardLogger('../logs', name = 'test', version = args.version),
    max_epochs=args.max_epochs,
    log_every_n_steps=1,
    accelerator = 'gpu' if torch.cuda.is_available() else 'cpu',
    callbacks = [
            pl.callbacks.TQDMProgressBar(refresh_rate = 1000)
            ],
    deterministic = True,
)

trainer.fit(model,  datamodule=datamodule)

Global seed set to 42
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/elias/anaconda3/envs/ribfrac/lib/python3.8/site-packages/lightning/pytorch/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(

  | Name    | Type             | Params
---------------------------------------------
0 | net     | DummyNetwork     | 67.1 M
1 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
67.1 M    Trainable params
0         Non-trainable params
67.1 M    Total params
268.453   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/elias/anaconda3/envs/ribfrac/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/Users/elias/anaconda3/envs/ribfrac/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 2: 100%|██████████| 2/2 [00:08<00:00,  4.02s/it, v_num=on_0, train_loss=-, train_acc=0.000, val_loss=0.000, val_acc=0.500]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 2/2 [00:08<00:00,  4.33s/it, v_num=on_0, train_loss=-, train_acc=0.000, val_loss=0.000, val_acc=0.500]
